In [ ]:
# To prevent the hassle of installing the package in the wrong environment, 
# run the following to install phillistine directly into the current environment.
# phillistine is a package developed by one of the authors of the Corcoran et al. (2018) paper. 
# this paper was followed to calculate the IAF scores.
import sys
print(sys.executable)
!{sys.executable} -m pip install --upgrade git+https://gitlab.com/palday/philistine.git

In [ ]:
import mne 
import pandas as pd
from philistine.mne import savgol_iaf
from pathlib import Path
import glob
import pyvista as pv
import numpy as np
import os
import math, numbers
from scipy.signal import savgol_filter

# Identify the folders where the data is stored
buddhist = Path("/Users/lucas.assen/Desktop/Master Thesis/Buddhists_preprocessed")
control = Path("/Users/lucas.assen/Desktop/Master Thesis/Controls_preprocessed")

# Load the converted FIF files
buddhist_fif_files = sorted(glob.glob(str(buddhist / "*.fif")))
control_fif_files = sorted(glob.glob(str(control / "*.fif")))

D_0 = {"Buddhists": {}, "Controls": {}}

def load_data_files(file_list, condition):
    for file in file_list:
        Participant_ID = Path(file).stem
        print(f"Loading {Participant_ID} ({condition})...")

        raw = mne.io.read_raw_fif(file, preload=True)
        D_0[condition][Participant_ID] = raw

load_data_files(buddhist_fif_files, "Buddhists")
load_data_files(control_fif_files, "Controls")

In [ ]:
# The eyes closed marker indicates the start and end of the eyes closed period in the eeg recording; it is used to extract and append the periods
# of eyes closed data in each participant.
Eyes_closed_marker = 102

# The channels refers to the relevant channels for the IAF calculation. These follow the channels used in Corcoran et al. (2018).
Channels = ['Pz','P1','P2','POz','PO3','PO4','Oz','O1','O2']

# The csv with the bad channels for each participant is loaded to help create a map for each part which shows the channels 
# that should be excluded from the IAF calculation.
BAD_CSV = "/Users/lucas.assen/Desktop/Master Thesis/bad_channel_summary_2.csv"
bad_df = pd.read_csv(BAD_CSV, sep=";", engine="python",
                     quoting=csv.QUOTE_NONE).fillna("")
bad_df.columns = bad_df.columns.str.strip().str.lower()

# Since the participant IDs did not line up in the csv and the fif file, the following defitnition creates 
# a standardised key for each participant to link up the bad channels with the correct participant.
def pid_key(s):
    base = re.match(r"^0*?(\d+)", str(s)).group(1)
    suff = "c" if "control" in str(s).lower() else ""
    return f"{base}{suff}"

bad_map = {pid_key(r["participant_id"]):
           {ch for ch in str(r["bad_ch_names"]).split(",") if ch}
           for _, r in bad_df.iterrows()}

# the following function checks if the value is a real number and finite, which is relevant for the savgol_iaf function.
def real_number(x):
    return isinstance(x, numbers.Real) and math.isfinite(x)

# The following function checks if the channel is in the list of channels used for the IAF calculation.
# It returns the index of the channel in the list of channels.
# This is used to select the relevant channels from the raw data.
def picks_channels(info, bads=set()):
    return [info.ch_names.index(ch)
            for ch in Channels if ch in info.ch_names and ch not in bads]

# The following function extracts the blocks of data between the eyes closed markers.
# It finds the markers in the raw data and crops the eyes closed sections to extract and append them.
def eyes_closed_blocks(raw, code=Eyes_closed_marker):
    events = mne.find_events(raw, 'Status', initial_event=True, shortest_event=0)
    sfreq = raw.info['sfreq']; idx = np.where(events[:, 2] == code)[0]
    blocks, start = [], None
    for ix in idx:
        smp = events[ix, 0]
        if start is None:
            start = smp
        else:
            blocks.append(raw.copy().crop(start/sfreq, smp/sfreq)); start = None
    return blocks

# the flattening happens through a log transformation of the psd which is the lineary fitted to yield the background 
# trend which is then reconstructed by removing the background from the original psd.
def flatten_psd(freqs, psd):
    mask = (freqs >= 1) & (freqs <= 35)
    x, y = np.log10(freqs[mask]), np.log10(psd[mask])
    m, c = np.polyfit(x, y, 1)
    bg   = 10 ** (m*np.log10(freqs) + c)
    return psd / bg

# the following function is created to accurately follow the process of calculating the IAF as presented in Corcoran et al. (2018).
# In the philistine package, tthe savgol_iaf function lacks the ability to calculate the Q value which is used to wheight each channel in the array for its 
# contribution to the personalised alpha frequency (PAF) calculation across all channels. According to the paper, the Q value
# is calculated by taking the second order derivative to find the inflections points of the PSD curve. These are used to integrate and calculate the area under the curve, 
# which is divided by the curves width, i.e, the frequency range or difference between the two inflection points, this results in the Q value.
def savgol_iaf_channel_Q_weight(raw, picks, fmin=7, fmax=13,
                                sg_order=5, sg_window=11):
    out, sfreq = {}, raw.info['sfreq']; n_fft = int(2*sfreq)
    for p in picks:
        ch_raw = raw.copy().pick(picks=[p])

        # First the standard savgol_iaf function is applied to get the PAF and CoG values.
        paf, cog, (f1, f2) = savgol_iaf(ch_raw, fmin=fmin, fmax=fmax)

        # To extract the Q values the proces of the paper is followed whereby the welch PSD is calculated
        spec = ch_raw.compute_psd(fmin=fmin, fmax=fmax, n_fft=n_fft, verbose=False)
        psd, freqs = spec.get_data()[0], spec.freqs
        
        # The psd is flattened to remove background noise/to normalise it and the savgol filter is applied using the same parameters as the savgol_iaf function
        psd_flat = flatten_psd(freqs, psd)
        psd_sg   = savgol_filter(psd_flat, sg_window, sg_order)

        # for locating the inflection point the PAF yielded by the savgol_iaf function is used to identify the peak of the PSD curve
        # alternatively the highest point of the PSD curve, which will be between 7 and 13 Hz, is used to identify the peak of the PSD curve.
        # then the second order derivative is calculated.
        idx_peak = (np.argmin(np.abs(freqs - paf))
                    if real_number(paf) else psd_sg.argmax())
        d2 = np.gradient(np.gradient(psd_sg))

        # The to find the inflection points the firest sign changes left and right of the peak are search for.
        i_left = idx_peak
        while i_left > 1 and np.sign(d2[i_left]) == np.sign(d2[i_left-1]):
            i_left -= 1
        i1 = freqs[i_left]

        i_right = idx_peak
        while (i_right < len(d2)-2 and
               np.sign(d2[i_right]) == np.sign(d2[i_right+1])):
            i_right += 1
        i2 = freqs[i_right]

        # the q value is then calculate by integrating the area under the curve between the two inflection points, which is then divided by the width of the curve, 
        # i.e., the difference between the two inflection points.
        if real_number(i1) and real_number(i2) and i2 > i1:
            band = (freqs >= i1) & (freqs <= i2)
            area = np.trapz(psd_sg[band], freqs[band])
            Q = area / (i2 - i1)
        else:
            Q = np.nan

        #The outout is the combined output of the savgol_iaf function and the Q value.
        out[ch_raw.ch_names[0]] = dict(PAF=paf, CoG=cog, Q=Q,
                                       f1=f1, f2=f2, i1=i1, i2=i2)
    return out

# In the next section all prior function are applied to the whole dictionary of participants and their respective blocks of data.
rows = []
for group, subs in D_0.items():
    for Part_ID, raw in subs.items():
        pid = Part_ID.split("_")[0]        
        bads = bad_map.get(pid, set())

        blocks = eyes_closed_blocks(raw)
        if not blocks:
            print(f"{Part_ID}: no Eyes-closed pairs");  continue

        per_block = []
        for k, block in enumerate(blocks, 1):
            picks = picks_channels(block.info, bads=bads)
            if not picks:
                print(f"{Part_ID}: all IAF channels bad in block {k}");  continue

            ch = savgol_iaf_channel_Q_weight(block, picks)

            PAF_Q = [(d['PAF'], d['Q']) for d in ch.values()
                     if real_number(d['PAF']) and real_number(d['Q'])]
            if PAF_Q:
                PAF_arr, Q_arr = map(np.array, zip(*PAF_Q))
                PAF_m = float((Q_arr/Q_arr.max() * PAF_arr).sum() / (Q_arr/Q_arr.max()).sum())
                Q_m   = float(Q_arr.mean())
            else:
                PAF_m = Q_m = np.nan

            CoGs = [d['CoG'] for d in ch.values() if real_number(d['CoG'])]
            CoG_m = float(np.mean(CoGs)) if CoGs else np.nan
            IAF_m = CoG_m if real_number(CoG_m) else PAF_m

            per_block.append((IAF_m, Q_m))
            rows.append(dict(group=group, id=Part_ID, block=k,
                             PAF_M=PAF_m, CoG_M=CoG_m,
                             IAF_M=IAF_m, Q_M=Q_m))

        valid = [v for v, _ in per_block if real_number(v)]
        if len(valid) > 1:
            rows.append(dict(group=group, id=Part_ID,
                             block='GA', IAF_GrandAvg=float(np.mean(valid))))
        else:
            print(f"{Part_ID}: no valid IAF values – GA skipped")

IAF_df = pd.DataFrame(rows)
print(IAF_df.head())

In [3]:
# Save the output to a CSV file
IAF_df.to_csv('IAF_After_channel_rejection.csv', index=False)

In [ ]:
import pandas as pd

# Load the IAF Grand Average values per participant
iaf_df = pd.read_csv('/Users/lucas.assen/Desktop/Master Thesis/Data/my_mne_project/mne_env/IAF_After_channel_rejection.csv')

bands_df = pd.DataFrame({
    'id':           iaf_df['id'],
    'delta_low':    0.3,
    'delta_high':   0.4 * iaf_df['IAF_GrandAvg'],
    'theta_low':    0.4 * iaf_df['IAF_GrandAvg'],
    'theta_high':   0.6 * iaf_df['IAF_GrandAvg'],
    'alpha_low':    0.6 * iaf_df['IAF_GrandAvg'],
    'alpha_high':   1.2 * iaf_df['IAF_GrandAvg'],
    'beta_low':     1.2 * iaf_df['IAF_GrandAvg'],
    'beta_high':    35.0,
})

bands = bands_df.dropna(how='any').reset_index(drop=True)
bands.to_csv('Individualised_freq_bands_after_chan_reject.csv', index=False)